In [11]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Set the path to the dataset
dataset_folder = "train"

# Initialize data and labels lists
data = []
labels = []

# Image processing parameters
image_size = (64, 64)  # Resize images to 64x64 pixels

# Load images and assign labels based on filenames
for img_name in os.listdir(dataset_folder):
    img_path = os.path.join(dataset_folder, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
    img = cv2.resize(img, image_size)  # Resize image
    img = img.flatten()  # Flatten the image to a 1D array
    data.append(img)

   
    # Assign label based on filename
    if "cat" in img_name:
        labels.append("cat")
    elif "dog" in img_name:
        labels.append("dog")



In [13]:
# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Encode labels to integers (cats=0, dogs=1)
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [15]:
pca = PCA(n_components=50)  # Adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


In [17]:
# Train the LinearSVC model
svm = LinearSVC(random_state=42)
svm.fit(X_train_pca, y_train)

C:\Users\Abel\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\Abel\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [19]:
y_pred = svm.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.50


In [21]:
# Example prediction on a new image
test_image_path = "test1"
test_img = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
test_img = cv2.resize(test_img, image_size)
test_img = test_img.flatten().reshape(1, -1)
test_img_pca = pca.transform(test_img)
prediction = svm.predict(test_img_pca)
print(f"Predicted class: {le.inverse_transform(prediction)[0]}")

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
